In [380]:
import re

text = "6787lkjlkj"

# Ищем слова, которые НЕ начинаются с буквы "H"
pattern = re.compile(r'[0-9]+')

matches = pattern.findall(text)

print(matches)

['6787']


In [470]:



import re

def isParenthesesBalanced(expression):
    stack = []
    opening_brackets = {'(', '[', '{'}
    closing_brackets = {')': '(', ']': '[', '}': '{'}

    for char in expression:
        if char in opening_brackets:
            stack.append(char)
        elif char in closing_brackets:
            if not stack or stack.pop() != closing_brackets[char]:
                return False
    return not stack  # Все скобки должны быть правильно закрыты

pattern = r'^\s*((([a-zA-Z_]+)([a-zA-Z0-9_]*)\s*=\s*)|\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*[-+*/]\s*)+\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*;'



def check_assignment(expression):
    return re.fullmatch(pattern, expression)
prevMatch = re.match(r'.+;', """dfdgf = rfg4rg + d4rt345yrt ;
if[d*2-3.5]{
	while[b-5]{
		a=a+2*c;
		b=b-1;
	}
}
write> a;""")
print(prevMatch)
match = check_assignment(prevMatch.group())
print(match)
matchedText = match.group()
print(matchedText)
if match :
    if not isParenthesesBalanced(match.group()):
        print(f"Error scoup.")
    else:
        print(check_assignment("dfdgf = (rfg4rg + d4rt345yrt;"))
else:
     print("not matches")

<re.Match object; span=(0, 29), match='dfdgf = rfg4rg + d4rt345yrt ;'>
<re.Match object; span=(0, 29), match='dfdgf = rfg4rg + d4rt345yrt ;'>
dfdgf = rfg4rg + d4rt345yrt ;
<re.Match object; span=(0, 29), match='dfdgf = (rfg4rg + d4rt345yrt;'>


In [1]:
import re

def clearBreakCharacter(expression):
	if expression.startswith('\n') :
		expression = expression[1:] 
		return clearBreakCharacter(expression)
	if expression.endswith('\n'):
		expression = expression[:-1]
		return clearBreakCharacter(expression)
	return expression

# провірка на парність відкритих і закритих дужок 
def isParenthesesBalanced(expression):
    stack = []
    opening_brackets = {'(', '[', '{'}
    closing_brackets = {')': '(', ']': '[', '}': '{'}

    for char in expression:
        if char in opening_brackets:
            stack.append(char)
        elif char in closing_brackets:
            if not stack or stack.pop() != closing_brackets[char]:
                return False
    return not stack  # Все скобки должны быть правильно закрыты

def findCloseBlock(str, index, num):
	indexOpen = str.find('{', index + 1)
	indexClose = str.find('}', index + 1)
	# print(index)
	# print(indexOpen)
	# print(indexClose)
	# Якщо немає парних закриваючих скобок 
	if indexClose == -1:
		raise ValueError("Немає закриваючих скобок")
	# якщо попереду немає відкриваючих вложених скобок або вони є дальше за перші закриваючі  і закриваючі скобки є дальше по індексу як відкиваючі 
	elif (indexOpen == -1 or indexOpen > indexClose) and indexClose > index:
		if num == 0:
			# Повертаємо номер закриваючої дужки
			return indexClose
		else:
			num = num - 1
			return findCloseBlock(str, indexClose, num)

	#Якщо попереду є ще одні відкриваючі скобки 
	elif indexOpen < indexClose:
		num = num + 1
		return findCloseBlock(str, indexOpen, num)


def separateText(list, str):
	str = clearBreakCharacter(str)
	# print(str)
	# print('--------------+++-------------')

	# пошук команд read|write
	match = re.match(r'\s*(read|write).*[\n;]', str)
	if match:
		matched_text = match.group()
		matched_text1= clearBreakCharacter(matched_text)
		# Вырезаем найденный текст из начала строки, учитывая пробелы
		modified_text = str[len(matched_text):]
		list.append(matched_text1)
		# print(modified_text)
		# print('---------------------------')
		return separateText(list, modified_text)
	# пошук команд if|ifnot|while|whilenot
	match = re.match(r'\s*(if|ifnot|while|whilenot).*\n', str)
	if match:
		indexCloseBlock = findCloseBlock(str, str.find('{'), 0)
		modified_text = str[indexCloseBlock+2:]
		# print('***************')
		# print(modified_text)
		# print('***************')



		matched_text = str[:indexCloseBlock+1]
		list.append([matched_text])
		return separateText(list, modified_text)


	# пошук на вирази формула
	# print('***** ',str)
	paternPrev = r'.+;'
	pattern = r'^\s*((([a-zA-Z_]+)([a-zA-Z0-9_]*)\s*=\s*)|\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*[-+*/]\s*)+\s*\(*\s*(([a-zA-Z_]+)([a-zA-Z0-9_]*)|([0-9]+))\s*\)*\s*;'
	matchPrev = re.match(paternPrev, str)# Шукаємо строку яка закінчується символом ;
	if matchPrev:
		match = re.fullmatch(pattern, matchPrev.group())#Тепер в ній провіряємо справність формули
		# print('---', match)
		if match:
			matched_text = match.group()
			if not isParenthesesBalanced(matched_text):#Перевірка на парність закриваючих і відкриваючих дужок
				print(f"Error scoup.")
			else:
				list.append(matched_text)
				modified_text = re.sub(pattern, '', str)
				# print('***************+++++++')
				# print(modified_text)
				# print('***************+++++++')
				return separateText(list, modified_text)

	# else:
		# if len(str) > 0:
		# 	print("Sysntasis error")

	return list

program_text = """
read> a;
read> b;
dfdgf = rfg4rg + d4rt345yrt;
read> c;
dfdgf = rfg4rg + d4rt345yrt;
dfdgf = rfg4rg + d4rt345yrt;
if[d*2-3.5]{
	while[b-5]{
		a=a+2*c;
		b=b-1;
	}
}
write> a;"""



lists = separateText([], program_text)
print(len(lists))
print(lists)

for stroka in lists:
	print(stroka)




8
['read> a;', 'read> b;', 'dfdgf = rfg4rg + d4rt345yrt;', 'read> c;', 'dfdgf = rfg4rg + d4rt345yrt;', 'dfdgf = rfg4rg + d4rt345yrt;', ['if[d*2-3.5]{\n\twhile[b-5]{\n\t\ta=a+2*c;\n\t\tb=b-1;\n\t}\n}'], 'write> a;']
read> a;
read> b;
dfdgf = rfg4rg + d4rt345yrt;
read> c;
dfdgf = rfg4rg + d4rt345yrt;
dfdgf = rfg4rg + d4rt345yrt;
['if[d*2-3.5]{\n\twhile[b-5]{\n\t\ta=a+2*c;\n\t\tb=b-1;\n\t}\n}']
write> a;


In [ ]:
# indexClose = findCloseBlock(program_text, program_text.find('{'), 0)
# print(findCloseBlock(program_text, program_text.find('{'), 0))
# print(program_text.find('hhh'))




text = "read cars are popular, and Ford is a well-known brand."

# Using re.match to check if the text starts with "ford"
match_result = re.match(r'^\s*(read|write|if|ifnot|while|whilenot)', text)

# if match_result:
#     matched_word = match_result.group(1)
#     print(f"The text starts with '{matched_word}'.")
# else:
#     print("The text does not start with 'read', 'write', or 'if'.")


# Разбиваем текст программы на отдельные строки по символу ;
lines = program_text.split('\n')
index = program_text.find('}', 82)
# print(index)

array_of_command = []
new_person = {'name': 'Eve', 'age': 28, 'city': 'Seattle'}
# array_of_dicts.append(new_person)
# print(array_of_dicts)
# Выводим результат
n=0
# for line in lines:
#     # print(line)
#     if re.match(r'^\s*(read)', line):
#         print(line)
#     elif  re.match(r'^\s*(write)', line):
#         print(line)
#     elif  re.match(r'^\s*(if)', line):
#         index = line.find('if')
#         print(line)
#     elif  re.match(r'^\s*(ifnot)', line):
#         index = line.find('ifnot')
#         print(line)
#     elif  re.match(r'^\s*(while)', line):
#         index = line.find('while')
#         print(line)
#     elif  re.match(r'^\s*(whilenot)', line):
#         index = line.find('whilenot')
#         print(line)
#     else:
#         print('expre') 
#     # print('---')
# print(re.match(r'\s*(read|write|if)(;?)', 'read > a ;'))

In [343]:
import re


class VirtualMachine:
    def __init__(self, code):
        self.code = code
        self.variables = {}
        self.stack = []

    def execute(self):
        label_stack = []
        for i, instruction in enumerate(self.code):
            if instruction.startswith('READ'):
                var = instruction.split()[1][:-1]
                value = input(f"Enter value for {var}: ")
                self.variables[var] = float(value)
            elif instruction.startswith('PUSH'):
                value = float(instruction.split()[1])
                self.stack.append(value)
            elif instruction in {'+', '-', '*', '/'}:
                if self.stack:
                    operand2 = self.stack.pop()
                    if self.stack:
                        operand1 = self.stack.pop()
                        result = self.perform_operation(operand1, operand2, instruction)
                        self.stack.append(result)
            elif instruction.startswith('STORE'):
                var = instruction.split()[1]
                if self.stack:
                    value = self.stack.pop()
                    self.variables[var] = value
            elif instruction.startswith('COMPARE'):
                if self.stack:
                    operand2 = self.stack.pop()
                    if self.stack:
                        operand1 = self.stack.pop()
                        result = self.compare(operand1, operand2, instruction)
                        self.stack.append(result)
            elif instruction.startswith('GOTOIFNOT'):
                label = instruction.split()[1]
                if self.stack and not self.stack.pop():
                    label_stack.append(label)
            elif instruction.startswith('GOTO'):
                label = instruction.split()[1]
                self.jump_to_label(label)
            elif instruction.startswith('true_label'):
                if self.stack:
                    label_stack.pop()



    def perform_operation(self, operand1, operand2, operation):
        if operation == '+':
            return operand1 + operand2
        elif operation == '-':
            return operand1 - operand2
        elif operation == '*':
            return operand1 * operand2
        elif operation == '/':
            return operand1 / operand2

    def compare(self, operand1, operand2, comparison):
        if comparison == '<':
            return operand1 < operand2
        elif comparison == '>':
            return operand1 > operand2
        elif comparison == '==':
            return operand1 == operand2

    def jump_to_label(self, label):
        pass  # Implement jump logic

# Example usage





def lex(code):
    # Tokenize the input code
    # tokens = re.findall(r'\bifnot\b|\bif\b|\bwhile\b|\bwrite\b|\bread\b|[a-zA-Z_]\w*|[-+*/=<>]|[0-9]+|\S', code)
    tokens = re.findall(r'\bifnot\b|\bif\b|\bwhile\b|\bwrite\b|\bread\b|[a-zA-Z_]\w*|[-+*/=<>]|[0-9]+|\S', code)

    print(tokens)
    return tokens

def parse(tokens):
    # Parse the tokens and generate intermediate code
    commands = []
    variables = set()

    for token in tokens:
        if re.match(r'[a-zA-Z_]\w*', token):
            variables.add(token)
        elif re.match(r'[-+*/=<>]|[0-9]+', token):
            commands.append(token)
    print(commands)
    print(variables)
    return commands, variables


def compile_program(code):
    tokens = lex(code)
    commands, variables = parse(tokens)
    
    compiled_code = []
    var_count = len(variables)
    label_count = 0
    
    for command in commands:
        if re.match(r'[a-zA-Z_]\w*', command):
            compiled_code.append(f'READ {command}')
        elif re.match(r'\d+', command):
            compiled_code.append(f'PUSH {command}')
        elif command in {'+', '-', '*', '/'}:
            compiled_code.append(f'{command}')
        elif command == '=':
            var_count += 1
            compiled_code.append(f'STORE t{var_count}')
        elif command in {'<', '>', '=='}:
            compiled_code.append(f'COMPARE {command}')
        elif command == 'if':
            label_count += 1
            compiled_code.append(f'GOTOIFNOT true_label_{label_count}')
        elif command == 'ifnot':
            label_count += 1
            compiled_code.append(f'GOTO true_label_{label_count}')
    return compiled_code

# Example usage
program = """
read> a;
read> b;
read> c;
d=c*a;
if[d*2-3.5]{
    while[b-5]{
        a=a+2*c; 
        b=b-1;
    }
}
write> a;
"""

program = """
readi> a=5;
relkjkj;lad> b;
read> c;
"""

compiled_code = compile_program(program)
print(compiled_code)
vm = VirtualMachine(compiled_code)
print(vm.execute())






['readi', '>', 'a', '=', '5', ';', 'relkjkj', ';', 'lad', '>', 'b', ';', 'read', '>', 'c', ';']
['>', '=', '5', '>', '>']
{'c', 'lad', 'relkjkj', 'readi', 'a', 'b', 'read'}
['COMPARE >', 'STORE t8', 'PUSH 5', 'COMPARE >', 'COMPARE >']
None
